## Homework 1

#### Michelle Appel (10170359)
#### Nils Hulzeboch (10749411)
#### Yves van Montfort (XXX)

#### 11-01-2018

### Theoretical Part [15 pts]

1. Hypothesis Testing – The problem of multiple comparisons [ 5 points ]
Experimentation in AI often happens like this:
A. Modify/Build an algorithm
B. Compare the algorithm to a baseline by running a hypothesis test.
C. If not significant, go back to step A
D. If significant, start writing a paper.
Compute the probabilities below  How many hypothesis tests, m, does it take to get to  (with Type I error for each test = α):
(a) P(mt  h  experiment gives significant result | m experiments lacking power to reject H 0)  ? (b) P(at least one significant result | m experiments lacking power to reject H 0)  ?

2. Bias and unfairness in Interleaving experiments [ 10 points ]
Balance interleaving has been shown to be biased in a number of corner cases. An example was given during the lecture with two ranked lists of length 3 being interleaved, and a randomly clicking population of users that resulted in algorithm A winning 2⁄3 of the time, even though in theory the percentage of wins should be 50% for both algorithms. Can you come up with a
situation of two ranked lists of length 3 and a distribution of clicks over them for which Team-draft interleaving is unfair to the better algorithm?

### Experimental Part [85 pts]